In [10]:
import os
import numpy as np
import pandas as pd
from scipy.linalg import orth
class PPCA(object):

    def __init__(self):

        self.raw = None
        self.data = None
        self.C = None
        self.means = None
        self.stds = None
        self.eig_vals = None

    def _standardize(self, X):

        if self.means is None or self.stds is None:
            raise RuntimeError("Fit model first")

        return (X - self.means) / self.stds

    def fit(self, data, d=None, tol=1e-4, min_obs=10, verbose=False):

        self.raw = data
        self.raw[np.isinf(self.raw)] = np.max(self.raw[np.isfinite(self.raw)])

        valid_series = np.sum(~np.isnan(self.raw), axis=0) >= min_obs

        data = self.raw[:, valid_series].copy()
        N = data.shape[0]
        D = data.shape[1]

        self.means = np.nanmean(data, axis=0)
        self.stds = np.nanstd(data, axis=0)

        data = self._standardize(data)
        observed = ~np.isnan(data)
        missing = np.sum(~observed)
        data[~observed] = 0

        # initial

        if d is None:
            d = data.shape[1]

        if self.C is None:
            C = np.random.randn(D, d)
        else:
            C = self.C
        CC = np.dot(C.T, C)
        X = np.dot(np.dot(data, C), np.linalg.inv(CC))
        recon = np.dot(X, C.T)
        recon[~observed] = 0
        ss = np.sum((recon - data) ** 2) / (N * D - missing)

        v0 = np.inf
        counter = 0

        while True:

            Sx = np.linalg.inv(np.eye(d) + CC / ss)

            # e-step
            ss0 = ss
            if missing > 0:
                proj = np.dot(X, C.T)
                data[~observed] = proj[~observed]
            X = np.dot(np.dot(data, C), Sx) / ss

            # m-step
            XX = np.dot(X.T, X)
            C = np.dot(np.dot(data.T, X), np.linalg.pinv(XX + N * Sx))
            CC = np.dot(C.T, C)
            recon = np.dot(X, C.T)
            recon[~observed] = 0
            ss = (np.sum((recon - data) ** 2) + N * np.sum(CC * Sx) + missing * ss0) / (N * D)

            # calc diff for convergence
            det = np.log(np.linalg.det(Sx))
            if np.isinf(det):
                det = abs(np.linalg.slogdet(Sx)[1])
            v1 = N * (D * np.log(ss) + np.trace(Sx) - det) \
                 + np.trace(XX) - missing * np.log(ss0)
            diff = abs(v1 / v0 - 1)
            if verbose:
                print
                diff
            if (diff < tol) and (counter > 5):
                break

            counter += 1
            v0 = v1

        C = orth(C)
        vals, vecs = np.linalg.eig(np.cov(np.dot(data, C).T))
        order = np.flipud(np.argsort(vals))
        vecs = vecs[:, order]
        vals = vals[order]

        C = np.dot(C, vecs)

        # attach objects to class
        self.C = C
        self.data = data
        self.eig_vals = vals
        self._calc_var()

    def transform(self, data=None):

        if self.C is None:
            raise RuntimeError('Fit the data model first.')
        if data is None:
            return np.dot(self.data, self.C)
        return np.dot(data, self.C)

    def _calc_var(self):

        if self.data is None:
            raise RuntimeError('Fit the data model first.')

        data = self.data.T

        # variance calc
        var = np.nanvar(data, axis=1)
        total_var = var.sum()
        self.var_exp = self.eig_vals.cumsum() / total_var

    def save(self, fpath):

        np.save(fpath, self.C)

    def load(self, fpath):

        assert os.path.isfile(fpath)

        self.C = np.load(fpath)


df = pd.read_csv('TestDataSet.csv')

na_df = df[df.isnull().any(axis=1)]
na_features = na_df.iloc[:,0:3]

data = df.as_matrix()

# reverse PCA on complete set
nComp = 3
ppca = PPCA()
ppca.fit(data=data, d=100, verbose=True)

variance_explained = ppca.var_exp
newdata = ppca.data*ppca.stds +ppca.means
model_params = ppca.C

component_mat = ppca.transform()

print(variance_explained)
print("Original row: ", data[1:13])
print("Estimated row: ", newdata[1:13])

for i in 
    sse = sum((newdata[:,i] - range(:,i)))^2


[0.65976375 0.84993506 0.95165745 1.00205339]
Original row:  [[7.185e+00 4.030e+00 1.780e+01 7.287e+05]
 [6.998e+00 2.940e+00 1.870e+01 7.014e+05]
 [7.147e+00 5.330e+00 1.870e+01 7.602e+05]
 [6.430e+00 5.210e+00 1.870e+01 6.027e+05]
 [      nan 1.243e+01 1.520e+01 4.809e+05]
 [6.172e+00 1.915e+01 1.520e+01 5.691e+05]
 [5.631e+00 2.993e+01       nan 3.465e+05]
 [6.004e+00 1.710e+01 1.520e+01 3.969e+05]
 [6.377e+00 2.045e+01 1.520e+01 3.150e+05]
 [6.009e+00       nan 1.520e+01 3.969e+05]
 [5.889e+00 1.571e+01 1.520e+01 4.557e+05]
 [5.949e+00 8.260e+00 2.100e+01 4.284e+05]]
Estimated row:  [[7.18500000e+00 4.03000000e+00 1.78000000e+01 7.28700000e+05]
 [6.99800000e+00 2.94000000e+00 1.87000000e+01 7.01400000e+05]
 [7.14700000e+00 5.33000000e+00 1.87000000e+01 7.60200000e+05]
 [6.43000000e+00 5.21000000e+00 1.87000000e+01 6.02700000e+05]
 [6.24535322e+00 1.24300000e+01 1.52000000e+01 4.80900000e+05]
 [6.17200000e+00 1.91500000e+01 1.52000000e+01 5.69100000e+05]
 [5.63100000e+00 2.99300000e